In [5]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from os import getenv
import dotenv
import time

In [3]:
dotenv.load_dotenv()
# Faiss
FAISS_REVIEWS_PATH_EUCLIDEAN = "faiss_index_euclidean"
FAISS_REVIEWS_PATH_COSINE = "faiss_index_cosine"
FAISS_INDEX_NAME = "index"
FAISS_DISTANCE_STRATEGY_EUCLIDEAN ='EUCLIDEAN_DISTANCE'
FAISS_DISTANCE_STRATEGY_COSINE = "COSINE_DISTANCE"
LLM_MODEL_NAME = "gemini-1.5-flash" #"gemini-pro"
EMBEDDING_MODEL_NAME = "models/text-embedding-004"
EMBEDDINGS_CACHE_STORE="./cache/"

GOOGLE_API_KEY = getenv('GOOGLE_API_KEY')

embedding_model = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL_NAME)
store = LocalFileStore(EMBEDDINGS_CACHE_STORE)
embedding_model = CacheBackedEmbeddings.from_bytes_store(embedding_model, store)


vector_db = FAISS.load_local(folder_path=FAISS_REVIEWS_PATH_COSINE,
                             embeddings=embedding_model,
                             index_name=FAISS_INDEX_NAME,
                             allow_dangerous_deserialization=True)

llm = ChatGoogleGenerativeAI(model=LLM_MODEL_NAME)

In [4]:
review_template_str = """
Your job is to use Google Map restaurants and bars reviews to help people find best places to go for a meal or a drink.
Use the following information and reviews to answer the questions. if the question is not about restaurants,
then kindly tell the user that you can only provide assistance and answer questions related to restaurants. if the user doesn't mention the city name,
always assume the user is asking about Padova.
If the context provided to you does not contain the answer of the question, tell the user that there is no answer in the reviews.
Answer context:
{context}
"""

system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_template_str
    )
)

human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [system_prompt, human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

reviews_retriever = vector_db.as_retriever(search_kwargs={'k': 20,})

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | llm
    | StrOutputParser()
)

In [4]:
def get_questions():
    with open("questions.txt", mode="r") as f:
        questions = []
        for line in f:
            questions.append(line.split("\n")[0])
    return questions

questions = get_questions()

In [6]:
with open("Question-Answer.txt", mode="w") as f:
    for i, question in enumerate(questions):
        f.write(str(i) + ": " + question + "\n" + review_chain.invoke(question) + "\n")
        time.sleep(1)

In [7]:
extra_questions = ["Give me information of some good restaurants in Via Roma, Padova.",
                   "What restaurants are in Piazza Camillo Benso Conte di Cavour?"
                   ]

In [8]:
with open("Question-Answer-cosine.txt", mode="r+") as f:
  # Write separator lines
  f.seek(0, 2)
  f.write("********************\n********************\n")
  for i, question in enumerate(extra_questions):
    # Move the pointer to the end of the file
    f.seek(0, 2)  
    f.write(str(i) + ": " + question + "\n" + review_chain.invoke(question) + "\n")
    time.sleep(1)
